In [1]:
# Write some markdown to explain that this notebook will be mainly used for the capstone project.
# Import the pandas library as pd.
# Import the Numpy library as np.
# Print the following the statement: Hello Capstone Project Course!

# Coursera IBM Data Science Professional Capstone Project
<p>
This notebook contains the capstone project that fulfills part of the requirement of the IBM Data Science Professional Course offered by Coursera

In [2]:
import pandas as pd
import numpy as np
print ("Hello Captone Project Course!")

Hello Captone Project Course!


In [3]:
# The code was removed by Watson Studio for sharing.

## Part 1 Web Scraping Toronto Neighbourhoods
This section below scrapes the wikipedia website for the Toronto Neighbourhoods

In [4]:
# we shall use BeautifulSoup to scrape the website
import requests
from bs4 import BeautifulSoup

web = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(web)

soup = BeautifulSoup(page.content, "html.parser")
table = soup.find('table')
rows = table.find_all('tr')

#print number of rows read
print("Table consists of ", len(rows), " rows")

Table consists of  181  rows


In [5]:
# First row are headers
hlist = []
for h in rows[0].find_all('th'):
    t = h.text.split('\n') # remove the newline at the end
    hlist.append(t[0])
hlist

['Postal Code', 'Borough', 'Neighborhood']

In [6]:
# create a DataFrame and populate with the data
tbl_list = []

idx = 0
for r in rows[1:]:
    clist = []
    for c in r.find_all('td'):
        clist.append(c.text.split('\n')[0])
    t={hlist[0]:clist[0], hlist[1]:clist[1], hlist[2]:clist[2]}
    tbl_list.append(t)
       
df = pd.DataFrame(tbl_list) # somehow cannot append dataframes one at a time
#re-order the columns
df = df[hlist]
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
# clean out the dataframes
print('Number of unassigned boroughs: ', (df['Borough']== 'Not assigned').sum())

df1 = df[df['Borough']!= 'Not assigned']
print('After filtering', df1.shape)


Number of unassigned boroughs:  77
After filtering (103, 3)


Perform sanity check that no unassigned neighborhoods and no duplicate postal code. It seems the wiki has since been cleaned out of such errors.

In [8]:
print("Check that no neighborhoods are unassigned:", (df1['Neighborhood']=='Not assigned').sum())

print("Check following postal codes are not unique")
pc = (df1['Postal Code'].value_counts())
for t in range(0, pc.shape[0]):
    if pc[t]>1:
        print(pc.index[t])
    #endif
#endfor


Check that no neighborhoods are unassigned: 0
Check following postal codes are not unique


In [9]:
df1.set_index('Postal Code', inplace=True)
print('Answer for question 1: Shape of the dataframe is: ', df1.shape)
df1.head()

Answer for question 1: Shape of the dataframe is:  (103, 2)


,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Part 2 Adding the Longitude and Latitude
I choose to use the CSV data instead of the unreliable geocoder.  We can always update the CSV file if necessary, though the neighborhoods are not likely to change with time.

In [10]:
dfll = pd.read_csv('http://cocl.us/Geospatial_data')

In [11]:
dfll.set_index('Postal Code', inplace=True)

In [12]:
df_result = pd.concat([df1, dfll], sort=True, axis=1)
df_result.head(10)

,Borough,Neighborhood,Latitude,Longitude
M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
M1G,Scarborough,Woburn,43.770992,-79.216917
M1H,Scarborough,Cedarbrae,43.773136,-79.239476
M1J,Scarborough,Scarborough Village,43.744734,-79.239476
M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## Part 3 Clustering the Neighborhoods

In [14]:
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
LIMIT = 100
df_toronto = df_result[df_result['Borough'].str.contains('Toronto')]
print(df_toronto.shape)

tor_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                             latitudes=df_toronto['Latitude'],
                             longitudes=df_toronto['Longitude'])


(39, 4)
The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Pl

In [17]:
# initial analysis of the data
c =tor_venues['Venue Category'].unique()
print('There are ', len(c), " unique categories")
t = tor_venues.groupby('Neighborhood').count()
t[['Venue Category']].describe()

There are  237  unique categories


,Venue Category
count,39.000000
mean,41.487179
std,33.456076
min,2.000000
25%,16.000000
50%,32.000000
75%,62.500000
max,100.000000


In [18]:
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
tor_onehot.insert(0,'Toronto Neighborhood', tor_venues['Neighborhood']) #need to change name because there is a category named "Neighbourhood"
tor_onehot.head(5)

,Toronto Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
tor_grouped = tor_onehot.groupby('Toronto Neighborhood').mean().reset_index()
tor_grouped

,Toronto Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.017857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.015385,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.012821,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012821,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.012821,0.012821,0.000000,0.025641
7,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,...,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Toronto Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Toronto Neighborhood'] = tor_grouped['Toronto Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(39)

,Toronto Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Seafood Restaurant,Restaurant,Café,Cheese Shop,Beer Bar,Lounge,Irish Pub
1,"Brockton, Parkdale Village, Exhibition Place",Café,Bakery,Breakfast Spot,Coffee Shop,Furniture / Home Store,Burrito Place,Convenience Store,Italian Restaurant,Stadium,Intersection
2,"Business reply mail Processing Centre, South C...",Yoga Studio,Skate Park,Auto Workshop,Brewery,Burrito Place,Butcher,Comic Shop,Farmers Market,Fast Food Restaurant,Garden
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Sculpture Garden,Plane,Boat or Ferry,Boutique,Bar,Harbor / Marina
4,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Sandwich Place,Japanese Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Department Store,Modern European Restaurant
5,Christie,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Baby Store,Italian Restaurant,Diner,Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Gay Bar,Café,Pub,Men's Store,Mediterranean Restaurant,Hotel
7,"Commerce Court, Victoria Hotel",Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant
8,Davisville,Sandwich Place,Dessert Shop,Café,Italian Restaurant,Gym,Coffee Shop,Sushi Restaurant,Pizza Place,Park,Pharmacy
9,Davisville North,Gym / Fitness Center,Hotel,Park,Food & Drink Shop,Sandwich Place,Department Store,Breakfast Spot,Gym,Pizza Place,Discount Store


In [22]:
# set number of clusters
kclusters = 5

tor_clustering = tor_grouped.drop('Toronto Neighborhood', 1)

tor_clustering.shape

(39, 237)

In [23]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters,init='random', n_init=50, random_state=35).fit(tor_clustering)

# check cluster labels generated for each row in the dataframe
# kmeans.labels_[0:10] 
pd.Series(kmeans.labels_).value_counts()

2    31
3     3
1     3
4     1
0     1
dtype: int64

As can be seen from the above there are 31 neighborhoods labeled "2".  This is not very ideal, as almost all the neighbourhoods are grouped in 1 cluster. Various initialization methods, runs, and random_state were tried, and this is about the best I could get. Nevertheless, I shall proceed with the labels as given for the moment.

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#neighborhoods_venues_sorted[neighborhoods_venues_sorted['Cluster Labels']==0].head()

# tor_merged = tor_data

# # merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
# tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# tor_merged.head() # check the last columns!

In [25]:
tor_merged = df_toronto[["Neighborhood", "Latitude", "Longitude"]]
tor_merged.rename(columns={'Neighborhood':'Toronto Neighborhood'}, inplace=True)
tor_merged.set_index('Toronto Neighborhood', inplace=True)


tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Toronto Neighborhood'), on='Toronto Neighborhood')
tor_merged.sort_values('Cluster Labels', inplace=True)
tor_merged.reset_index(inplace=True)


/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


<B>The labeled neighborhoods in Toronto are listed below</b>

In [26]:
tor_merged

,Toronto Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Roselawn,43.711695,-79.416936,0,Ice Cream Shop,Home Service,Garden,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
1,"High Park, The Junction South",43.661608,-79.464763,1,Café,Mexican Restaurant,Thai Restaurant,Bookstore,Arts & Crafts Store,Fast Food Restaurant,Cajun / Creole Restaurant,Flea Market,Speakeasy,Italian Restaurant
2,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Jewelry Store,Trail,Mexican Restaurant,Sushi Restaurant,Yoga Studio,Diner,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
3,Christie,43.669542,-79.422564,1,Grocery Store,Café,Park,Coffee Shop,Nightclub,Candy Store,Baby Store,Italian Restaurant,Diner,Restaurant
4,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,2,Coffee Shop,Aquarium,Café,Hotel,Brewery,Fried Chicken Joint,Restaurant,Scenic Lookout,Sporting Goods Shop,Italian Restaurant
5,"Commerce Court, Victoria Hotel",43.648198,-79.379817,2,Coffee Shop,Café,Restaurant,Hotel,Gym,American Restaurant,Japanese Restaurant,Italian Restaurant,Deli / Bodega,Seafood Restaurant
6,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,2,Sandwich Place,Café,Coffee Shop,Park,Burger Joint,Donut Shop,Indian Restaurant,Middle Eastern Restaurant,Pub,History Museum
7,"University of Toronto, Harbord",43.662696,-79.400049,2,Café,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Restaurant,Yoga Studio,Pub,Beer Bar
8,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,2,Café,Coffee Shop,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Mexican Restaurant,Gaming Cafe,Bar,Bakery,Park,Dessert Shop
9,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,2,Airport Service,Airport Lounge,Airport Terminal,Rental Car Location,Sculpture Garden,Plane,Boat or Ferry,Boutique,Bar,Harbor / Marina


<B>Plotting the clusters on the map</b>

In [27]:
# create map
# Use Toronto coordinates 43.6532° N, 79.3832° W
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Toronto Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters